<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection

Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.\
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [21]:
# provide your solution here
# Establish database connection
library(RODBC)
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            # e.g. "bludb"
dsn_hostname <- "6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" # e.g "54a2f15b-5c0f-46df-8954-.databases.appdomain.cloud"
dsn_port <- "30376"   # e.g. "32733" 
dsn_protocol <- "TCPIP"            # i.e. "TCPIP"
dsn_uid <- "frl39174"        # e.g. "zjh17769"
dsn_pwd <- "TBaKhLbDxl1lkatq"      # e.g. "zcwd4+8gbq9bm5k4"  
dsn_security <- "ssl"

conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path)

    sql.info <- sqlTypeInfo(conn)
    conn.info <- odbcGetInfo(conn)
    conn.info["DBMS_Name"]
    conn.info["DBMS_Ver"]
    conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0700"

Driver_ODBC_Ver 
        "03.51"

ERROR: Error in odbcGetTables(conn): could not find function "odbcGetTables"


## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [2]:
query <- paste("SELECT COUNT(*) AS COUNT FROM SEOUL_BIKE_SHARING")
seoul_bike_count <- sqlQuery(conn, query)
seoul_bike_count

,COUNT
,<int>
1,8465


## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [3]:
# provide your solution here
query2 <- paste("SELECT SUM(HOUR) FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT<>0")
total_hrs <- sqlQuery(conn, query2)
total_hrs

,1
,<int>
1,97407


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [4]:
# provide your solution here
query3 <- paste("SELECT * FROM SEOUL_BIKE_SHARING LIMIT 1")
weather_Q <- sqlQuery(conn,query3)
weather_Q

,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>
1,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0,0,0,Winter,No Holiday,Yes


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [5]:
# provide your solution here
query4 <- paste("SELECT DISTINCT SEASONS FROM SEOUL_BIKE_SHARING")
season_q <- sqlQuery(conn,query4)
season_q

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [6]:
# provide your solution here
query5 <- paste("SELECT MIN(DATE), MAX(DATE) FROM SEOUL_BIKE_SHARING")
date_q <- sqlQuery(conn,query5)
date_q

,1,2
,<fct>,<fct>
1,01/01/2018,31/12/2017


## Task 6 - Subquery - 'all-time high'

#### determine which date and hour had the most bike rentals.


### Solution 6


In [7]:
# provide your solution here
query6 <- paste("SELECT DATE, HOUR FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT = (SELECT MAX(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING)")
most_q <- sqlQuery(conn,query6)
most_q

,DATE,HOUR
,<fct>,<int>
1,19/06/2018,18


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [8]:
# provide your solution here
query7 <- paste("SELECT HOUR, SEASONS, AVG(TEMPERATURE) AS HOURLY_TEMP, AVG(RENTED_BIKE_COUNT) AS HOURLY_RENTED FROM SEOUL_BIKE_SHARING
GROUP BY HOUR, SEASONS ORDER BY AVG(RENTED_BIKE_COUNT) DESC LIMIT 10")
hourly_q <- sqlQuery(conn,query7)
hourly_q

,HOUR,SEASONS,HOURLY_TEMP,HOURLY_RENTED
,<int>,<fct>,<dbl>,<int>
1,18,Summer,29.38696,2135
2,18,Autumn,16.03086,1983
3,19,Summer,28.27283,1889
4,20,Summer,27.06630,1801
5,21,Summer,26.27826,1754
6,18,Spring,15.97222,1689
7,22,Summer,25.69891,1567
8,17,Autumn,17.27778,1562
9,17,Summer,30.07500,1526


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.


### Solution 8


In [9]:
# provide your solution here
query8 <- paste("SELECT HOUR, AVG(RENTED_BIKE_COUNT) AS HOURLY_RENTED, MIN(RENTED_BIKE_COUNT) AS MIN, MAX(RENTED_BIKE_COUNT) AS MAX, STDDEV(RENTED_BIKE_COUNT) AS STD, 
SEASONS FROM SEOUL_BIKE_SHARING
GROUP BY HOUR, SEASONS ORDER BY AVG(RENTED_BIKE_COUNT) DESC LIMIT 10")
rental_q <- sqlQuery(conn, query8)
rental_q

,HOUR,HOURLY_RENTED,MIN,MAX,STD,SEASONS
,<int>,<int>,<int>,<int>,<dbl>,<fct>
1,18,2135,17,3556,884.0829,Summer
2,18,1983,40,3298,778.4414,Autumn
3,19,1889,18,2984,728.8799,Summer
4,20,1801,10,2579,662.2163,Summer
5,21,1754,17,2505,596.1374,Summer
6,18,1689,22,3251,898.8971,Spring
7,22,1567,16,2309,516.6434,Summer
8,17,1562,23,2432,554.3165,Autumn
9,17,1526,25,2664,608.7917,Summer


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [10]:
query9 <- paste("SELECT AVG(RENTED_BIKE_COUNT) AS AVG_BIKE,
AVG(TEMPERATURE) AS AVG_TEMP,
AVG(HUMIDITY) AS AVG_HUMID,
AVG(WIND_SPEED) AS AVG_WIND,
AVG(VISIBILITY) AS AVG_VISIB,
AVG(DEW_POINT_TEMPERATURE) AS AVG_DEW,
AVG(SOLAR_RADIATION) AS AVG_SOLAR,
AVG(RAINFALL) AS AVG_RAINFALL,
AVG(SNOWFALL) AS AVG_SNOWFALL,
SEASONS FROM SEOUL_BIKE_SHARING GROUP BY SEASONS ORDER BY AVG(RENTED_BIKE_COUNT) DESC")
weather_seaq <- sqlQuery(conn, query9)
weather_seaq

# provide your solution here

,AVG_BIKE,AVG_TEMP,AVG_HUMID,AVG_WIND,AVG_VISIB,AVG_DEW,AVG_SOLAR,AVG_RAINFALL,AVG_SNOWFALL,SEASONS
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1034,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000,Summer
2,924,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026,Autumn
3,746,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000,Spring
4,225,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000,Winter


## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [25]:
# provide your solution here
query10 <- paste("SELECT BS.BICYCLES, WC.CITY_ASCII, WC.COUNTRY, WC.LAT, WC.LNG, WC.POPULATION FROM BIKE_SHARING_SYSTEMS AS BS, 
WORLD_CITIES AS WC WHERE WC.CITY_ASCII = BS.CITY AND WC.CITY = 'Seoul'")
total_bike <- sqlQuery(conn, query10)
total_bike

,BICYCLES,CITY_ASCII,COUNTRY,LAT,LNG,POPULATION
,<int>,<fct>,<fct>,<dbl>,<dbl>,<int>
1,20000,Seoul,"Korea, South",37.58,127,21794000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [30]:
# provide your solution here
query11 <- ("SELECT BS.BICYCLES, WC.CITY_ASCII, WC.COUNTRY, WC.LAT, WC.LNG, WC.POPULATION FROM BIKE_SHARING_SYSTEMS AS BS, 
WORLD_CITIES AS WC WHERE WC.CITY_ASCII = BS.CITY AND BICYCLES BETWEEN 15000 AND 20000 ORDER BY BICYCLES DESC")
city_names <- sqlQuery(conn, query11)
city_names

,BICYCLES,CITY_ASCII,COUNTRY,LAT,LNG,POPULATION
,<int>,<fct>,<fct>,<dbl>,<dbl>,<int>
1,20000,Seoul,"Korea, South",37.58,127.00,21794000
2,20000,Weifang,China,36.71,119.10,9373000
3,20000,Xi'an,China,34.26,108.90,7135000
4,20000,Zhuzhou,China,27.84,113.14,3855609
5,19165,Shanghai,China,31.16,121.46,22120000
6,16000,Beijing,China,39.90,116.39,19433000
7,15000,Ningbo,China,29.87,121.54,7639000


In [31]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                  |
| ---------- | ------- | ------------- | ----------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                            |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review           |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
